# pyoctree introduction
---
Requirements:
* pyoctree
* vtk >= 6.2.0

In [1]:
# Imports
from __future__ import print_function
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import sys, vtk
sys.path.append('../')
import pyoctree
from pyoctree import pyoctree as ot

In [2]:
print('pyoctree version = ', pyoctree.__version__)
print('vtk version = ', vtk.vtkVersion.GetVTKVersion())

pyoctree version =  0.2.5
vtk version =  7.1.0


## Load 3D model geometry (stl file)

In [4]:
# Read in stl file using vtk
reader = vtk.vtkSTLReader()
reader.SetFileName("pyoctree\Examples\knot.stl")
reader.MergingOn()
reader.Update()
stl = reader.GetOutput()
print("Number of points    = %d" % stl.GetNumberOfPoints())
print("Number of triangles = %d" % stl.GetNumberOfCells())

Number of points    = 38214
Number of triangles = 76428


In [5]:
# Extract polygon info from stl

# 1. Get array of point coordinates
numPoints   = stl.GetNumberOfPoints()
pointCoords = np.zeros((numPoints,3),dtype=float)
for i in range(numPoints):
    pointCoords[i,:] = stl.GetPoint(i)
    
# 2. Get polygon connectivity
numPolys     = stl.GetNumberOfCells()
connectivity = np.zeros((numPolys,3),dtype=np.int32)
for i in range(numPolys):
    atri = stl.GetCell(i)
    ids = atri.GetPointIds()
    for j in range(3):
        connectivity[i,j] = ids.GetId(j)

In [6]:
# Show format of pointCoords
pointCoords

array([[ 2.54317427,  7.41368246,  2.8599999 ],
       [ 2.51169538,  7.40434027,  2.8599999 ],
       [ 2.48648334,  7.48027468,  2.8599999 ],
       ..., 
       [ 1.20494771,  8.17073154,  2.8599999 ],
       [ 1.20671427,  8.17605209,  2.8599999 ],
       [ 1.20778596,  8.18155479,  2.8599999 ]])

In [7]:
# Show format of connectivity
connectivity

array([[    0,     1,     2],
       [    2,     3,     0],
       [    3,     4,     0],
       ..., 
       [38182, 38181, 38180],
       [38190, 38189, 38187],
       [38189, 38188, 38187]])

## Generate octree

In [8]:
# Create octree structure containing stl poly mesh
tree = ot.PyOctree(pointCoords,connectivity)

In [9]:
# Print out basic Octree data
print("Size of Octree               = %.3fmm" % tree.root.size)
print("Number of Octnodes in Octree = %d" % tree.getNumberOfNodes())
print("Number of polys in Octree    = %d" % tree.numPolys)

Size of Octree               = 3.115mm
Number of Octnodes in Octree = 2497
Number of polys in Octree    = 76428


## Find intersections between 3D object and ray

In [10]:
# Create a list containing a single ray
xs,xe,ys,ye,zs,ze = stl.GetBounds()
x = 0.5*np.mean([xs,xe])
y = np.mean([ys,ye])
rayPointList = np.array([[[x,y,zs],[x,y,ze]]],dtype=np.float32)

In [11]:
# Find if an intersection occurred
for i in tree.rayIntersections(rayPointList):
    print(i==1)

True


In [17]:
# Get intersection points for a single ray
ray = rayPointList[0]
for i in tree.rayIntersection(ray):
    print('Intersected tri = %d,' % i.triLabel, 'Intersection coords = [%.2f, %.2f, %.2f]' % tuple(i.p), ',  Parametric. dist. along ray = %.2f' % i.s)

Intersected tri = 73584, Intersection coords = [0.75, 7.33, 3.32] ,  Parametric. dist. along ray = 0.46
Intersected tri = 70341, Intersection coords = [0.75, 7.33, 3.64] ,  Parametric. dist. along ray = 0.78
Intersected tri = 32425, Intersection coords = [0.75, 7.33, 3.87] ,  Parametric. dist. along ray = 1.01
Intersected tri = 29843, Intersection coords = [0.75, 7.33, 4.42] ,  Parametric. dist. along ray = 1.56


In [20]:
# Get list of intersected triangles
triLabelList = [i.triLabel for i in tree.rayIntersection(ray)]
triLabelList

[73584, 70341, 32425, 29843]

In [22]:
# Get tris
triList = [tree.polyList[i] for i in triLabelList]
triList

[<PyTri 73584>, <PyTri 70341>, <PyTri 32425>, <PyTri 29843>]

In [34]:
# Convert ray from start/end points into unit vector
from numpy.linalg import norm
rayVect = ray[1]-ray[0]
rayVect /= norm(rayVect)
rayVect

array([ 0.,  0.,  1.], dtype=float32)

In [38]:
# Find if tri face normal is in the same direction as the ray
for tri in triList:
    if np.dot(tri.N,rayVect)>0:
        print("Tri %d face normal is in the same direction as ray i.e. This is an exit point." % tri.label)
    else:
        print("Tri %d face normal is in the opposite direction as the ray i.e. This is an entry point" % tri.label)

Tri 73584 face normal is in the opposite direction as the ray i.e. This is an entry point
Tri 70341 face normal is in the same direction as ray i.e. This is an exit point.
Tri 32425 face normal is in the opposite direction as the ray i.e. This is an entry point
Tri 29843 face normal is in the same direction as ray i.e. This is an exit point.
